*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep = '\t')
test_df = pd.read_csv(test_file_path, sep = '\t')

In [ ]:
tam = train_df['ham'].value_counts()[0]-train_df['ham'].value_counts()[1]
ham_df1 = train_df[train_df['ham']=='ham']
spam_df1 = train_df[train_df['ham']=='spam']
ham_df1 = ham_df1.drop(ham_df1.index[:tam])
train_df = ham_df1.append(spam_df1).reset_index(drop=True)

tam = train_df['ham'].value_counts()[0]-test_df['ham'].value_counts()[1]
ham_df2 = test_df[test_df['ham']=='ham']
spam_df2 = test_df[test_df['ham']=='spam']
ham_df2 = ham_df2.drop(ham_df2.index[:tam])
test_df = ham_df2.append(spam_df2).reset_index(drop=True)



In [ ]:
train_dataframe = train_df.copy()
train_labels = train_dataframe.pop('ham').astype('category').cat.codes
test_dataframe = test_df.copy()
test_labels = test_dataframe.pop('ham').astype('category').cat.codes

In [ ]:
train_dataframe.columns = ['msg']
test_dataframe.columns = ['msg']

In [ ]:
# train_series = train_dataframe['msg'].str.lower().str.replace('[^\w\s]','').str.split()
# test_series = test_dataframe['msg'].str.lower().str.replace('[^\w\s]','').str.split()
train_series = train_dataframe['msg']
test_series = test_dataframe['msg']

In [ ]:
#train_dataframe = train_dataframe.drop('msg', 1).join(train_dataframe.msg.str.join('|').str.get_dummies())
#test_dataframe = test_dataframe.drop('msg', 1).join(test_dataframe.msg.str.join('|').str.get_dummies())

In [ ]:
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words = vocab_size, char_level=False, oov_token=oov_tok)
tokenizer.fit_on_texts(train_series)

In [ ]:
training_sequences = tokenizer.texts_to_sequences(train_series)
training_padded = keras.preprocessing.sequence.pad_sequences(training_sequences, maxlen = max_len, padding = padding_type,
                                                             truncating = trunc_type)
testing_sequences = tokenizer.texts_to_sequences(test_series)
testing_padded = keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen = max_len, padding = padding_type,
                                                            truncating = trunc_type)

In [ ]:
embeding_dim = 16
drop_value = 0.2
n_dense = 24

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dropout(drop_value))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
history = model.fit(training_padded, train_labels, epochs=30, validation_data=(testing_padded, test_labels))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  tk_txt = tokenizer.texts_to_sequences([pred_text])
  pad_txt = keras.preprocessing.sequence.pad_sequences(tk_txt, maxlen = max_len, padding = padding_type,
                                                             truncating = trunc_type)
  if(round((model.predict(pad_txt))[0][0]) == 1):  
    return [(model.predict(pad_txt))[0][0], 'spam']
  return [(model.predict(pad_txt))[0][0], 'ham']

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
